![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [36]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

In [37]:
# joining datasets 
df = brands.merge(finance, on="product_id").merge(info, on="product_id").merge(reviews, on="product_id")

#dropping null values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3117 entries, 1 to 3178
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     3117 non-null   object 
 1   brand          3117 non-null   object 
 2   listing_price  3117 non-null   float64
 3   sale_price     3117 non-null   float64
 4   discount       3117 non-null   float64
 5   revenue        3117 non-null   float64
 6   product_name   3117 non-null   object 
 7   description    3117 non-null   object 
 8   rating         3117 non-null   float64
 9   reviews        3117 non-null   float64
dtypes: float64(6), object(4)
memory usage: 267.9+ KB


In [38]:
# listing price quartiles
price_labels = ["Budget", "Average", "Expensive", "Elite"]
df["price_label"] = pd.qcut(df["listing_price"], q=4, labels=price_labels)

# grouping by brand and price label
adidas_vs_nike = df.groupby(["brand", "price_label"], as_index=False).agg(num_products=("price_label", "count"), mean_revenue = ("revenue", "mean")).round(2)
adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33
5,Nike,Average,8,675.59
6,Nike,Expensive,47,500.56
7,Nike,Elite,130,1367.45


In [39]:
# product description lengths
df["description_length"] = df["description"].str.len()
print(df["description_length"].max())

# categorizing description lengths
limits = [0, 100, 200, 300, 400, 500, 600, 700]
labels = ["100", "200", "300", "400", "500", "600", "700"]
df["description_length"] = pd.cut(df["description_length"], bins = limits, labels = labels)

# description lengths vs rating
description_lengths = df.groupby("description_length", as_index=False).agg(mean_rating = ("rating", "mean"), num_reviews = ("reviews", "count")).round(2)
description_lengths

687


,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.19,526
2,300,3.28,1785
3,400,3.29,651
4,500,3.35,118
5,600,3.12,15
6,700,3.65,15
